In [1]:
import uuid
from googleads import adwords
import sys

In [2]:
from csv import reader
import pandas as pd
import numpy as np

In [59]:
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#client_2 = ad_manager.AdManagerClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [4]:
# ID of your customer here
CUSTOMER_SERVICE_ID = '5922380045'

# Load customer account access
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

In [308]:
# Display which field is able to query for certain report type
REPORT_TYPE = 'CAMPAIGN_PERFORMANCE_REPORT'

def get_report_fields(client, report_type):
    # Initialize appropriate service.
    report_definition_service = client.GetService(
        'ReportDefinitionService', version='v201809')

    # Get report fields.
    fields = report_definition_service.getReportFields(report_type)

    # Display results.
    print('Report type "%s" contains the following fields:' % report_type)
    for field in fields:
        print(' - %s(%s)' % (field['fieldName'], field['fieldType']))
#         if 'enumValues' in field:
#             print('  := [%s]' % ', '.join(field['enumValues']))

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    get_report_fields(adwords_client, REPORT_TYPE)

Report type "CAMPAIGN_PERFORMANCE_REPORT" contains the following fields:
 - Period(BudgetPeriod)
 - UrlCustomParameters(CustomParameters)
 - AbsoluteTopImpressionPercentage(Double)
 - AccountCurrencyCode(String)
 - AccountDescriptiveName(String)
 - AccountTimeZone(String)
 - ActiveViewCpm(Money)
 - ActiveViewCtr(Double)
 - ActiveViewImpressions(Long)
 - ActiveViewMeasurability(Double)
 - ActiveViewMeasurableCost(Money)
 - ActiveViewMeasurableImpressions(Long)
 - ActiveViewViewability(Double)
 - AdNetworkType1(AdNetworkType1)
 - AdNetworkType2(AdNetworkType2)
 - AdvertisingChannelSubType(AdvertisingChannelSubType)
 - AdvertisingChannelType(AdvertisingChannelType)
 - AllConversionRate(Double)
 - AllConversions(Double)
 - AllConversionValue(Double)
 - Amount(Money)
 - AverageCost(Money)
 - AverageCpc(Money)
 - AverageCpe(Double)
 - AverageCpm(Money)
 - AverageCpv(Double)
 - AverageFrequency(Double)
 - AveragePageviews(Double)
 - AveragePosition(Double)
 - AverageTimeOnSite(Double)
 - Base

In [306]:
CAMPAIGN_FIELDS = ['ExternalCustomerId','CampaignId','AdvertisingChannelType', 'CampaignStatus',
                   'BiddingStrategyType','Amount','StartDate','EndDate','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
class DatePreset:
    today = 'TODAY'
    yesterday = 'YESTERDAY'
    lifetime = 'ALL_TIME'

def get_campaign_insights(client, campaign_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    report = {
        'reportName': 'CAMPAIGN_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': CAMPAIGN_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
                }
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(CAMPAIGN_FIELDS)),
            columns=CAMPAIGN_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])] = df[df.columns.difference(
        ['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference(['ExternalCustomerId', 'CampaignId', 'AdvertisingChannelType', 'CampaignStatus', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])].div(1000000)
    df[['StartDate','EndDate']] = df[['StartDate','EndDate']].apply( pd.to_datetime, errors='coerce' )
    return df

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    campaign_insights = get_campaign_insights(client, campaign_id=1442531270)


In [307]:
campaign_insights.info()
campaign_insights

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
ExternalCustomerId        1 non-null int64
CampaignId                1 non-null int64
AdvertisingChannelType    1 non-null object
CampaignStatus            1 non-null object
BiddingStrategyType       1 non-null object
Amount                    1 non-null float64
StartDate                 1 non-null datetime64[ns]
EndDate                   0 non-null datetime64[ns]
Cost                      1 non-null float64
AverageCost               1 non-null float64
Impressions               1 non-null int64
Clicks                    1 non-null int64
Conversions               1 non-null float64
AverageCpc                1 non-null float64
CostPerConversion         1 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(4), object(3)
memory usage: 200.0+ bytes


,ExternalCustomerId,CampaignId,AdvertisingChannelType,CampaignStatus,BiddingStrategyType,Amount,StartDate,EndDate,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,5922380045,1442531270,Display,enabled,cpc,500.0,2018-06-16,NaT,82716.846564,5.321122,5784947,15545,29.0,5.321122,2852.305054


In [304]:
ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus',
                  'CpmBid','CpvBid', 'CpcBid', 'TargetCpa', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_adgroup_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_adgroup_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'ADGROUP_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'ADGROUP_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': ADGROUP_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(ADGROUP_FIELDS)),
            columns=ADGROUP_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_adgroup_insights(client)
    adgroup_insights = get_adgroup_insights(client, campaign_id=1442531270)

In [305]:
adgroup_insights.info()
adgroup_insights

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 17 columns):
ExternalCustomerId     4 non-null int64
CampaignId             4 non-null int64
AdGroupType            4 non-null object
AdGroupId              4 non-null int64
AdGroupStatus          4 non-null object
CpmBid                 0 non-null float64
CpvBid                 0 non-null float64
CpcBid                 4 non-null float64
TargetCpa              0 non-null float64
BiddingStrategyType    4 non-null object
Cost                   4 non-null float64
AverageCost            4 non-null float64
Impressions            4 non-null float64
Clicks                 4 non-null float64
Conversions            4 non-null float64
AverageCpc             4 non-null float64
CostPerConversion      4 non-null float64
dtypes: float64(11), int64(3), object(3)
memory usage: 624.0+ bytes


,ExternalCustomerId,CampaignId,AdGroupType,AdGroupId,AdGroupStatus,CpmBid,CpvBid,CpcBid,TargetCpa,BiddingStrategyType,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,5922380045,1442531270,Display,59973178007,enabled,NaN,NaN,13.0,NaN,cpc,78867.936564,5.309899,5.434046,0.014853,29.0,5.309899,2719.584019
1,5922380045,1442531270,Display,59973178287,paused,NaN,NaN,9.0,NaN,cpc,567.890000,6.681059,0.046509,0.000085,0.0,6.681059,0.000000
2,5922380045,1442531270,Display,57122611117,paused,NaN,NaN,10.0,NaN,cpc,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,5922380045,1442531270,Display,59973178047,paused,NaN,NaN,9.0,NaN,cpc,3281.020000,5.405305,0.304392,0.000607,0.0,5.405305,0.000000


In [301]:
csv

,ExternalCustomerId,CampaignId,AdGroupType,AdGroupId,AdGroupStatus,CpmBid,CpvBid,CpcBid,BiddingStrategyType,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,5922380045,1442531270,Display,59973178007,enabled,NaN,NaN,13.0,cpc,78867.936564,5.309899,5.434046,0.014853,29.0,5.309899,2719.584019
1,5922380045,1442531270,Display,59973178287,paused,NaN,NaN,9.0,cpc,567.890000,6.681059,0.046509,0.000085,0.0,6.681059,0.000000
2,5922380045,1442531270,Display,57122611117,paused,NaN,NaN,10.0,cpc,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,5922380045,1442531270,Display,59973178047,paused,NaN,NaN,9.0,cpc,3281.020000,5.405305,0.304392,0.000607,0.0,5.405305,0.000000


In [49]:
import multiprocessing
# Timeout between retries in seconds.
BACKOFF_FACTOR = 5
# Maximum number of processes to spawn.
MAX_PROCESSES = multiprocessing.cpu_count()
# Maximum number of retries for 500 errors.
MAX_RETRIES = 5
# Maximum number of items to be sent in a single API response.
PAGE_SIZE = 100
def GetCustomerIDs(client):
    """Retrieves all CustomerIds in the account hierarchy.

    Note that your configuration file must specify a client_customer_id belonging
    to an AdWords manager account.

    Args:
    client: an AdWordsClient instance.
    Raises:
    Exception: if no CustomerIds could be found.
    Returns:
    A Queue instance containing all CustomerIds in the account hierarchy.
    """
    # For this example, we will use ManagedCustomerService to get all IDs in
    # hierarchy that do not belong to MCC accounts.
    managed_customer_service = client.GetService('ManagedCustomerService',
                                               version='v201809')

    offset = 0

    # Get the account hierarchy for this account.
    selector = {
      'fields': ['CustomerId'],
      'predicates': [{
          'field': 'CanManageClients',
          'operator': 'EQUALS',
          'values': [False]
      }],
      'paging': {
          'startIndex': str(offset),
          'numberResults': str(PAGE_SIZE)
      }
    }

    # Using Queue to balance load between processes.
    queue = multiprocessing.Queue()
    more_pages = True

    while more_pages:
        page = managed_customer_service.get(selector)
        print(page)
        if page and 'entries' in page and page['entries']:
            for entry in page['entries']:
                queue.put(entry['customerId'])
        else:
            raise Exception('Can\'t retrieve any customer ID.')
        offset += PAGE_SIZE
        selector['paging']['startIndex'] = str(offset)
        more_pages = offset < int(page['totalNumEntries'])

    return queue

In [53]:
ad_client = adwords.AdWordsClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')
GetCustomerIDs(ad_client)

{
    'totalNumEntries': 2,
    'Page.Type': 'ManagedCustomerPage',
    'entries': [
        {
            'name': None,
            'customerId': 8595344807,
            'canManageClients': None,
            'currencyCode': None,
            'dateTimeZone': None,
            'testAccount': None,
            'accountLabels': [],
            'excludeHiddenAccounts': None
        },
        {
            'name': None,
            'customerId': 4959531111,
            'canManageClients': None,
            'currencyCode': None,
            'dateTimeZone': None,
            'testAccount': None,
            'accountLabels': [],
            'excludeHiddenAccounts': None
        }
    ],
    'links': [
        {
            'managerCustomerId': 6017666929,
            'clientCustomerId': 8595344807,
            'linkStatus': None,
            'pendingDescriptiveName': None,
            'isHidden': False
        },
        {
            'managerCustomerId': 6017666929,
            'clientCustom

In [54]:
GetCustomerIDs(client)

{
    'totalNumEntries': 1,
    'Page.Type': 'ManagedCustomerPage',
    'entries': [
        {
            'name': None,
            'customerId': 5922380045,
            'canManageClients': None,
            'currencyCode': None,
            'dateTimeZone': None,
            'testAccount': None,
            'accountLabels': [],
            'excludeHiddenAccounts': None
        }
    ],
    'links': []
}
